In [1]:
import concurrent.futures
from ekonlpy import Mecab
import pandas as pd
import numpy as np
import re


## 데이터 확인

In [2]:
df = pd.read_csv(r'C:\Users\ran14\Workspace\BOK_workspace\preprocessing\News_U.csv')
pd.set_option('display.max_colwidth', None)
df

,Date,Title,Content,Link
0,2014.08.11,"<뉴욕환시-주간> 달러, 지정학적 불안에 조정 압박","이번 주(11~15일) 뉴욕 외환시장에서 달러화는 지정학적 불안으로 단기적으로 하락 압력을 받을 것으로 예상된다.지난 8일 (미국시간) 달러는 지정학적 불안 고조로 유로화와 엔화에 하락했다. 투자자들이 지정학적 이슈를 포지션 조정의 빌미로 삼은 때문이다.유로-달러는 뉴욕 후장에서 전장 대비 0.3% 오른 1.3410달러 근처에서 거래됐고, 달러-엔은 0.1% 떨어진 102.04엔에서 거래됐다.유로화는 과도한 달러 롱포지션 청산에 따른 숏커버링으로 달러에 대해 올랐다. 또 러시아와 우크라이나를 둘러싼 지정학적 긴장이 다소 완화된 점도 유로화 반등에 일조했다. 우크라이나 국경 근처에서 진행된 러시아의 군사훈련이 예정대로 끝났다는 소식에 불안이 완화됐다.그러나 지정학적 불안은 당분간 시장 참가자들이 주목할 이슈가 될 전망이다. 미국이 이라크에 대한 선별적 공습을 개시한 데다 가자지구를 둘러싼 불안도 지속되고 있어 투자자들이 지정학적 불안을 포지션 조정의 빌미로 삼을 수 있기 때문이다.버락 오바마 미국 대통령은 이라크 사태와 관련해 ""수주 안에 이 문제를 해결할 수 있을 것으로 보지 않으며 앞으로 시간이 좀 걸릴 것""이라고 언급해 이라크 개입 장기화 가능성을 예고했다.씨티그룹의 스티븐 잉글랜더 주요 10개국(G10) 외환 전략 헤드는 금융 매니저들이 고객들에게 구축한 포지션에서 나올 것을 권고해 지난 8일 '달러 매도-유로 매수' 움직임이 일었다고 설명했다.웨스턴 유니언의 조 마님보 선임 시장 애널리스트는 ""달러 강세가 차익 실현에 자리를 내줬다""고 말했다.그동안 투자자들이 달러화에 대해 과도하게 많은 롱포지션을 구축해온 때문이다.달러지수는 지난 6일에는 81.714까지 오르며 4주 연속 상승세로 마감했다. 지수는 최근 81.395까지 하락했으나 여전히 9개월래 최고치 근처에서 움직이고 있다. 애널리스트들은 지정학적 불안에 따른 포지션 조정이 단기에 그칠 것으로 예상했다.웨스트팩 뱅킹의 애널리스트들은 이번 지정학적 불안이 초기에 위험회피 심리를 일으켰지만, 하루 이틀만에 시장은 이를 넘겨버렸다고 말했다.이에 따라 이번 주 시장 참가자들은 3분기 국내총생산(GDP)을 가늠할 7월 소매판매 지표를 주목할 것으로 예상된다. 달러화가 강세를 재개하려면 미국 연방준비제도(Fed)의 조기 금리 인상 신호가 나타나야 하기 때문이다. 그러나 마님보 애널리스트는 ""시장에 연방준비제도(Fed)가 금리를 올리기까지 오랜 시간이 걸릴 것이라는 인식이 있다""고 전했다",https://news.einfomax.co.kr/news/articleView.html?idxno=116756
1,2014.08.11,8월 금통위 박빙 동결도 가능이주열 결단이 가른다③,한국은행 7월 금융통화위원회 의사록을 통해 나타난 6명의 금통위원이 본 경기판단은 전반적으로 비둘기(완화)적이라는데 이견이 없다 다만 비둘기적이었다고 해서 바로 인하로 돌아설 가능성은 낮아 보인다 일부 위원들 사이에서 최근 경제상황 변화와 중간에 한은이 단행한 금융중개지원대출 3조원 확대 등으로 굳이 금리인하에 손을 들지 않을 가능성도 있어 보이기 때문이다의사록을 보면 금리인하를 명백히 주장한 정해방 위원(F위원)은 선제적 대응 필요성을 강조하며 이달에도 인하에 손을 들 것으로 보인다 다만 한가지 짚어볼 점은 지난달 인하를 주장했던 그보다 동결에 손을 들었던 다른 일부 금통위원들에게서 경제를 보는 고민이 더 깊었다는 점이다 금통위 의사록을 보면 그는 세월호 사고의 영향이 일시적이고 제한적인 것인지 또는 장기적인 것인지를 예단하기는 어렵다며 최근 상황을 종합해 볼 때 경제주체들의 심리 위축이 장기화될 것으로 우려됨에 따라 현시점에서의 선제적인 경기대응이 필요한 것으로 판단된다고 밝혔을 뿐이다 14개월만에 나온 금리 인하 논거로는 왠지 두루뭉술한 측면이 커 그가 단지 정책조화 측면에서 인하에 손을 들었다고 밖에는 해석할 수 없는 대목이다 그렇잖아도 그는 기획예산처 차관 출신 인사인데다 기획재정부 추천 금통위원이다하성근 위원으로 추정되는 D위원도 이달 인하로 돌아설 것으로 보인다 그는 지난달 대부분의 시장참가자들이 이번 달 기준금리 동결을 기대하고 있다는 점도 고려해야 할 것이라고 밝혔다 이는 지난달 금리인하를 시사하는 듯한 논문발와 금통위 직전 한 세미나에 참석하면서 불러일으킨 논란 그리고 소수의견이 나온다면 하 위원일 것이라는 시장의 압도적 전망 등이 그를 위축시켰던 결과로밖에 해석되지 않는다 반대로 이달 기준금리 인하 기대감이 크다는 점을 고려해볼 때 하 위원은 부담없이 인하로 돌아설 것으로 보인다반면 문우식 위원으로 예상하는 E위원은 지난달에 이어 이달에도 동결 기조를 유지할 것으로 전망한다 그는 지난달 금통위에서 세월호 사고는 일시적인 충격으로 국내경제의 성장경로를 바꿀 정도는 아닌 것으로 판단된다며 일시적인 충격과 퍼머넌트 쇼크(permanent shock)를 구분할 필요가 있다고 밝힌 바 있다금통위원 사실상 3대3 갈릴듯이 총재 의지 따라 6대1 인하 or 4대3 동결문제는 장병화 정순원 함준호 위원이다 공개된 지난달 의사록만 봤을 때 다분히 중립에 가까운 비둘기로 보이기 때문이다우선 장병화 위원(부총재)으로 추정되는 C위원의 경우 성장경로나 GDP갭 축소 추세가 아직 꺾이지 않았다는 점 정책여력 확보를 강조하고 있다는 점에서 동결 가능성에 무게를 둔다 그렇잖아도 최근 전쟁이라는 글로벌 지정학적 리스크가 커지며 경제에 어떤 영향을 미칠지 모르는 상황이 전개되고 있다C위원을 장 부총재로 예상하는 이유는 이주열 총재가 지난달 금통위까지만 해도 금리인하 가능성에 상당한 거부감을 한 것으로 추정되기 때문이다 이 총재는 지난달 10일 금통위 기자회견에서 마이너스 GDP갭 성장세 둔화등 사실상 인하를 강력히 시사하는 문구들이 통화정책방향에 삽입된 것과 관련해 금리조정의 과거사례를 쭉 언급했는데 그런 과거사례를 염두에 둔 것은 전혀 아니다며 강하게 반발한 바 있다B위원은 금융중개지원대출에 만족한다면 동결에 손을 들겠지만 다양한 정책수단을 유연하게 발휘해야 한다는 점에서 금리인하도 염두했을수 있다 A위원은 최근 원화절상 압력이 급격히 줄었다는 점에서 일부 우려가 줄었을 수도 있겠다 하지만 세계경제 하방위험 저물가 지속 등은 여전히 인하에 무게를 둘 수 있는 변수다 다만 그는 지난달 비교적 담담하게 현 경제상황을 짚었었다결국 이들 세 위원의 판단은 이 총재의 결단에 크게 좌우될 수밖에 없어 보인다 그나마 동결 가능성이 있는 C위원이 장 부총재가 맞는다면 이 총재가 인하로 돌아설 경우 같은 집행부라는 점에서 인하로 돌아설 가능성이 높다 다만 그 역시 통화정책 전문가라는 점에서 꼭 총재와 세트로 판단하는 것은 잘못된 판단일수도 있음을 밝혀둔다어쨌든 이 총재가 인하로 돌아설 경우 이들 세 위원 역시 인하에 동참할 가능성이 높다 결국 6대1 인하가 이뤄질 것이다 반면 이 총재가 동결을 고수한다면 C위원으로 추정되는 부총재는 물론 A내지 B위원도 동결에 손을 들 것으로 보인다 이 경우 4대3 동결 가능성도 배제할 수 없겠다,https://www.edaily.co.kr/News/Read?newsId=01095526606186336&mediaCodeNo=257
2,2014.08.11,집 사라고 대출규제 풀었지만…집 안사고 생활비로 쓴다,정부가 주택시장 활성화를 위해 LTVDTI와 같은 대출규제를 풀었지만 정작 은행에선 정부의 정책 취지와는 다른 모습이 연출되고 있다 새로 은행 대출을 받아 집을 사겠다는 사람들의 발길은 뜸하지만 오히려 집을 담보로 대학 다니는 자식의 학자금을 마련하는 등 생활자금을 빌리려는 수요가 더 몰리고 있어서다 주택경기가 좀처럼 살아날 기미를 보이지 않다 보니 정부의 예상과 달리 주택담보대출을 받아 집을 사려는 수요 역시 움직이지 않고 있다는 게 금융권의 분석이다 문제는 정부가 정책 목적으로 내세운 주택시장 활성화와는 상관없이 확대된 주택담보대출이 자영업자 사업자금 등 생활자금으로 쓰일 개연성이 높아졌다는 점이다 빚에 빚을 더하는 셈인데 결국엔 가계부채에 악영향을 초래할

In [3]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93076 entries, 0 to 93075
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DATE      93076 non-null  object
 1   TITLE     93074 non-null  object
 2   CONTENTS  92939 non-null  object
 3   URL       93076 non-null  object
dtypes: object(4)
memory usage: 2.8+ MB
None
              DATE             TITLE          CONTENTS  \
count        93076             93074             92939   
unique        3455             90787             90862   
top     2015-12-17  1년 정기적금 최고금리 69%  이창용 한국은행 총재 기자회견   
freq           163                23                35   

                                                                                URL  
count                                                                         93076  
unique                                                                        93076  
top     https://www.edaily.co.kr/News/Read?newsId=02971686606186336&me

## 통합 데이터 Cleansing

In [10]:
def cleansing_text(text):
    # 종목 코드 (괄호 안 6자리 숫자) 제거
    text = re.sub(r'\(\d{6}\)', '', text)
    
    # 특수 기호 및 광고성 텍스트 제거
    text = re.sub(r'▶ 관련기사 ◀', '', text)
    text = re.sub(r'☞[^☞]*', '', text)
    
    # 맥락과 관련 없는 리스트 및 간추린 소식 제거
    text = re.sub(r'\d+\.\s[^<]*', '', text)
    text = re.sub(r'<간추린 소식>[^-]*', '', text)
    
    # 구두점, 따옴표, 기타 특수 문자 제거
    text = re.sub(r'[▲△▶▼▽◆◇=ㆍ/·.,;:!?\'"‘’“”~∼&()→%․\[\]\-–<>]', '', text)
    
    # '사진', '표'와 같은 텍스트 제거
    text = re.sub(r'사진|표', '', text)
    
    # 필요한 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [11]:
df['Title'] = df['Title'].apply(cleansing_text)
df['Content'] = df['Content'].apply(cleansing_text)

df_cleaned = df[['Date', 'Title', 'Content', 'Link']]
df_cleaned

,Date,Title,Content,Link
0,2014.08.11,뉴욕환시주간 달러 지정학적 불안에 조정 압박,이번 주1115일 뉴욕 외환시장에서 달러화는 지정학적 불안으로 단기적으로 하락 압력을 받을 것으로 예상된다지난 8일 미국시간 달러는 지정학적 불안 고조로 유로화와 엔화에 하락했다 투자자들이 지정학적 이슈를 포지션 조정의 빌미로 삼은 때문이다유로달러는 뉴욕 후장에서 전장 대비 03 오른 13410달러 근처에서 거래됐고 달러엔은 01 떨어진 10204엔에서 거래됐다유로화는 과도한 달러 롱포지션 청산에 따른 숏커버링으로 달러에 대해 올랐다 또 러시아와 우크라이나를 둘러싼 지정학적 긴장이 다소 완화된 점도 유로화 반등에 일조했다 우크라이나 국경 근처에서 진행된 러시아의 군사훈련이 예정대로 끝났다는 소식에 불안이 완화됐다그러나 지정학적 불안은 당분간 시장 참가자들이 주목할 이슈가 될 전망이다 미국이 이라크에 대한 선별적 공습을 개시한 데다 가자지구를 둘러싼 불안도 지속되고 있어 투자자들이 지정학적 불안을 포지션 조정의 빌미로 삼을 수 있기 때문이다버락 오바마 미국 대통령은 이라크 사태와 관련해 수주 안에 이 문제를 해결할 수 있을 것으로 보지 않으며 앞으로 시간이 좀 걸릴 것이라고 언급해 이라크 개입 장기화 가능성을 예고했다씨티그룹의 스티븐 잉글랜더 주요 10개국G10 외환 전략 헤드는 금융 매니저들이 고객들에게 구축한 포지션에서 나올 것을 권고해 지난 8일 달러 매도유로 매수 움직임이 일었다고 설명했다웨스턴 유니언의 조 마님보 선임 시장 애널리스트는 달러 강세가 차익 실현에 자리를 내줬다고 말했다그동안 투자자들이 달러화에 대해 과도하게 많은 롱포지션을 구축해온 때문이다달러지수는 지난 6일에는 81714까지 오르며 4주 연속 상승세로 마감했다 지수는 최근 81395까지 하락했으나 여전히 9개월래 최고치 근처에서 움직이고 있다 애널리스트들은 지정학적 불안에 따른 포지션 조정이 단기에 그칠 것으로 예상했다웨스트팩 뱅킹의 애널리스트들은 이번 지정학적 불안이 초기에 위험회피 심리를 일으켰지만 하루 이틀만에 시장은 이를 넘겨버렸다고 말했다이에 따라 이번 주 시장 참가자들은 3분기 국내총생산GDP을 가늠할 7월 소매판매 지를 주목할 것으로 예상된다 달러화가 강세를 재개하려면 미국 연방준비제도Fed의 조기 금리 인상 신호가 나타나야 하기 때문이다 그러나 마님보 애널리스트는 시장에 연방준비제도Fed가 금리를 올리기까지 오랜 시간이 걸릴 것이라는 인식이 있다고 전했다,https://news.einfomax.co.kr/news/articleView.html?idxno=116756
1,2014.08.11,8월 금통위 박빙 동결도 가능이주열 결단이 가른다③,한국은행 7월 금융통화위원회 의사록을 통해 나타난 6명의 금통위원이 본 경기판단은 전반적으로 비둘기완화적이라는데 이견이 없다 다만 비둘기적이었다고 해서 바로 인하로 돌아설 가능성은 낮아 보인다 일부 위원들 사이에서 최근 경제상황 변화와 중간에 한은이 단행한 금융중개지원대출 3조원 확대 등으로 굳이 금리인하에 손을 들지 않을 가능성도 있어 보이기 때문이다의사록을 보면 금리인하를 명백히 주장한 정해방 위원F위원은 선제적 대응 필요성을 강조하며 이달에도 인하에 손을 들 것으로 보인다 다만 한가지 짚어볼 점은 지난달 인하를 주장했던 그보다 동결에 손을 들었던 다른 일부 금통위원들에게서 경제를 보는 고민이 더 깊었다는 점이다 금통위 의사록을 보면 그는 세월호 사고의 영향이 일시적이고 제한적인 것인지 또는 장기적인 것인지를 예단하기는 어렵다며 최근 상황을 종합해 볼 때 경제주체들의 심리 위축이 장기화될 것으로 우려됨에 따라 현시점에서의 선제적인 경기대응이 필요한 것으로 판단된다고 밝혔을 뿐이다 14개월만에 나온 금리 인하 논거로는 왠지 두루뭉술한 측면이 커 그가 단지 정책조화 측면에서 인하에 손을 들었다고 밖에는 해석할 수 없는 대목이다 그렇잖아도 그는 기획예산처 차관 출신 인사인데다 기획재정부 추천 금통위원이다하성근 위원으로 추정되는 D위원도 이달 인하로 돌아설 것으로 보인다 그는 지난달 대부분의 시장참가자들이 이번 달 기준금리 동결을 기대하고 있다는 점도 고려해야 할 것이라고 밝혔다 이는 지난달 금리인하를 시사하는 듯한 논문발와 금통위 직전 한 세미나에 참석하면서 불러일으킨 논란 그리고 소수의견이 나온다면 하 위원일 것이라는 시장의 압도적 전망 등이 그를 위축시켰던 결과로밖에 해석되지 않는다 반대로 이달 기준금리 인하 기대감이 크다는 점을 고려해볼 때 하 위원은 부담없이 인하로 돌아설 것으로 보인다반면 문우식 위원으로 예상하는 E위원은 지난달에 이어 이달에도 동결 기조를 유지할 것으로 전망한다 그는 지난달 금통위에서 세월호 사고는 일시적인 충격으로 국내경제의 성장경로를 바꿀 정도는 아닌 것으로 판단된다며 일시적인 충격과 퍼머넌트 쇼크permanent shock를 구분할 필요가 있다고 밝힌 바 있다금통위원 사실상 3대3 갈릴듯이 총재 의지 따라 6대1 인하 or 4대3 동결문제는 장병화 정순원 함준호 위원이다 공개된 지난달 의사록만 봤을 때 다분히 중립에 가까운 비둘기로 보이기 때문이다우선 장병화 위원부총재으로 추정되는 C위원의 경우 성장경로나 GDP갭 축소 추세가 아직 꺾이지 않았다는 점 정책여력 확보를 강조하고 있다는 점에서 동결 가능성에 무게를 둔다 그렇잖아도 최근 전쟁이라는 글로벌 지정학적 리스크가 커지며 경제에 어떤 영향을 미칠지 모르는 상황이 전개되고 있다C위원을 장 부총재로 예상하는 이유는 이주열 총재가 지난달 금통위까지만 해도 금리인하 가능성에 상당한 거부감을 한 것으로 추정되기 때문이다 이 총재는 지난달 10일 금통위 기자회견에서 마이너스 GDP갭 성장세 둔화등 사실상 인하를 강력히 시사하는 문구들이 통화정책방향에 삽입된 것과 관련해 금리조정의 과거사례를 쭉 언급했는데 그런 과거사례를 염두에 둔 것은 전혀 아니다며 강하게 반발한 바 있다B위원은 금융중개지원대출에 만족한다면 동결에 손을 들겠지만 다양한 정책수단을 유연하게 발휘해야 한다는 점에서 금리인하도 염두했을수 있다 A위원은 최근 원화절상 압력이 급격히 줄었다는 점에서 일부 우려가 줄었을 수도 있겠다 하지만 세계경제 하방위험 저물가 지속 등은 여전히 인하에 무게를 둘 수 있는 변수다 다만 그는 지난달 비교적 담담하게 현 경제상황을 짚었었다결국 이들 세 위원의 판단은 이 총재의 결단에 크게 좌우될 수밖에 없어 보인다 그나마 동결 가능성이 있는 C위원이 장 부총재가 맞는다면 이 총재가 인하로 돌아설 경우 같은 집행부라는 점에서 인하로 돌아설 가능성이 높다 다만 그 역시 통화정책 전문가라는 점에서 꼭 총재와 세트로 판단하는 것은 잘못된 판단일수도 있음을 밝혀둔다어쨌든 이 총재가 인하로 돌아설 경우 이들 세 위원 역시 인하에 동참할 가능성이 높다 결국 6대1 인하가 이뤄질 것이다 반면 이 총재가 동결을 고수한다면 C위원으로 추정되는 부총재는 물론 A내지 B위원도 동결에 손을 들 것으로 보인다 이 경우 4대3 동결 가능성도 배제할 수 없겠다,https://www.edaily.co.kr/News/Read?newsId=01095526606186336&mediaCodeNo=257
2,2014.08.11,집 사라고 대출규제 풀었지만…집 안사고 생활비로 쓴다,정부가 주택시장 활성화를 위해 LTVDTI와 같은 대출규제를 풀었지만 정작 은행에선 정부의 정책 취지와는 다른 모습이 연출되고 있다 새로 은행 대출을 받아 집을 사겠다는 사람들의 발길은 뜸하지만 오히려 집을 담보로 대학 다니는 자식의 학자금을 마련하는 등 생활자금을 빌리려는 수요가 더 몰리고 있어서다 주택경기가 좀처럼 살아날 기미를 보이지 않다 보니 정부의 예상과 달리 주택담보대출을 받아 집을 사려는 수요 역시 움직이지 않고 있다는 게 금융권의 분석이다 문제는 정부가 정책 목적으로 내세운 주택시장 활성화와는 상관없이 확대된 주택담보대출이 자영업자 사업자금 등 생활자금으로 쓰일 개연성이 높아졌다는 점이다 빚에 빚을 더하는 셈인데 결국엔 가계부채에 악영향을 초래할 수 있다는 지적이 나온다 최근 금융당국은 이런 점을 우려해 시중은행 여신 담당 임원들을 소집해 될 수 있으면 확대된 주택담보대출이 정책

In [12]:
df = df_cleaned

## 결측치 확인 및 제거

In [3]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242832 entries, 0 to 242831
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Date     242832 non-null  object
 1   Title    242832 non-null  object
 2   Content  242782 non-null  object
 3   Link     242832 non-null  object
dtypes: object(4)
memory usage: 7.4+ MB
None
              Date                Title           Content  \
count       242832               242832            242782   
unique        3654               234119            240519   
top     2015.12.17  연합뉴스 이 시각 헤드라인 0800  이창용 한국은행 총재 기자회견   
freq           384                  343                35   

                                                                  Link  
count                                                           242832  
unique                                                          242832  
top     https://news.einfomax.co.kr/news/articleView.html?idxno=116756  
freq               

In [5]:
print(df.isnull().sum())
count_NaN = df[df.isnull().any(axis=1)]
count_NaN

Date        0
Title       0
Content    50
Link        0
dtype: int64


,Date,Title,Content,Link
10952,2015.01.15,RP 14일물 6조7일물 115조 매각,NaN,https://news.einfomax.co.kr/news/articleView.html?idxno=135727
12044,2015.01.29,RP 14일물 2027일물 200에 총 16조 낙찰,NaN,https://news.einfomax.co.kr/news/articleView.html?idxno=137445
13959,2015.02.26,RP 14일물 202에 6조 낙찰…응찰 733조,NaN,https://news.einfomax.co.kr/news/articleView.html?idxno=140670
15335,2015.03.12,RP 14일물 177에 6조 낙찰7일물 11조 모집,NaN,https://news.einfomax.co.kr/news/articleView.html?idxno=142426
15489,2015.03.13,러시아 기준금리 14로 1포인트 인하1보,NaN,https://www.yna.co.kr/view/AKR20150313178700009?section=search
16922,2015.03.26,RP 14일물 177에 6조7일물 10조 매각,NaN,https://news.einfomax.co.kr/news/articleView.html?idxno=144096
19401,2015.04.23,RP14일물 177에 6조7일물 11조 매각,NaN,https://news.einfomax.co.kr/news/articleView.html?idxno=150649
22205,2015.05.28,RP 14일물 177에 6조원 낙찰…7일물 11조 낙찰,NaN,https://news.einfomax.co.kr/news/articleView.html?idxno=161729
38183,2015.11.26,RP14일물 151에 6조 낙찰…7일물 95조 낙찰,NaN,https://news.einfomax.co.kr/news/articleView.html?idxno=197040
41145,2015.12.24,RP14일물 151에 5조 낙찰…7일물 95조 낙찰,NaN,https://news.einfomax.co.kr/news/articleView.html?idxno=201376


In [6]:
df_droped_na = df.dropna()
df_droped_na

,Date,Title,Content,Link
0,2014.08.11,뉴욕환시주간 달러 지정학적 불안에 조정 압박,이번 주1115일 뉴욕 외환시장에서 달러화는 지정학적 불안으로 단기적으로 하락 압력을 받을 것으로 예상된다지난 8일 미국시간 달러는 지정학적 불안 고조로 유로화와 엔화에 하락했다 투자자들이 지정학적 이슈를 포지션 조정의 빌미로 삼은 때문이다유로달러는 뉴욕 후장에서 전장 대비 03 오른 13410달러 근처에서 거래됐고 달러엔은 01 떨어진 10204엔에서 거래됐다유로화는 과도한 달러 롱포지션 청산에 따른 숏커버링으로 달러에 대해 올랐다 또 러시아와 우크라이나를 둘러싼 지정학적 긴장이 다소 완화된 점도 유로화 반등에 일조했다 우크라이나 국경 근처에서 진행된 러시아의 군사훈련이 예정대로 끝났다는 소식에 불안이 완화됐다그러나 지정학적 불안은 당분간 시장 참가자들이 주목할 이슈가 될 전망이다 미국이 이라크에 대한 선별적 공습을 개시한 데다 가자지구를 둘러싼 불안도 지속되고 있어 투자자들이 지정학적 불안을 포지션 조정의 빌미로 삼을 수 있기 때문이다버락 오바마 미국 대통령은 이라크 사태와 관련해 수주 안에 이 문제를 해결할 수 있을 것으로 보지 않으며 앞으로 시간이 좀 걸릴 것이라고 언급해 이라크 개입 장기화 가능성을 예고했다씨티그룹의 스티븐 잉글랜더 주요 10개국G10 외환 전략 헤드는 금융 매니저들이 고객들에게 구축한 포지션에서 나올 것을 권고해 지난 8일 달러 매도유로 매수 움직임이 일었다고 설명했다웨스턴 유니언의 조 마님보 선임 시장 애널리스트는 달러 강세가 차익 실현에 자리를 내줬다고 말했다그동안 투자자들이 달러화에 대해 과도하게 많은 롱포지션을 구축해온 때문이다달러지수는 지난 6일에는 81714까지 오르며 4주 연속 상승세로 마감했다 지수는 최근 81395까지 하락했으나 여전히 9개월래 최고치 근처에서 움직이고 있다 애널리스트들은 지정학적 불안에 따른 포지션 조정이 단기에 그칠 것으로 예상했다웨스트팩 뱅킹의 애널리스트들은 이번 지정학적 불안이 초기에 위험회피 심리를 일으켰지만 하루 이틀만에 시장은 이를 넘겨버렸다고 말했다이에 따라 이번 주 시장 참가자들은 3분기 국내총생산GDP을 가늠할 7월 소매판매 지를 주목할 것으로 예상된다 달러화가 강세를 재개하려면 미국 연방준비제도Fed의 조기 금리 인상 신호가 나타나야 하기 때문이다 그러나 마님보 애널리스트는 시장에 연방준비제도Fed가 금리를 올리기까지 오랜 시간이 걸릴 것이라는 인식이 있다고 전했다,https://news.einfomax.co.kr/news/articleView.html?idxno=116756
1,2014.08.11,8월 금통위 박빙 동결도 가능이주열 결단이 가른다③,한국은행 7월 금융통화위원회 의사록을 통해 나타난 6명의 금통위원이 본 경기판단은 전반적으로 비둘기완화적이라는데 이견이 없다 다만 비둘기적이었다고 해서 바로 인하로 돌아설 가능성은 낮아 보인다 일부 위원들 사이에서 최근 경제상황 변화와 중간에 한은이 단행한 금융중개지원대출 3조원 확대 등으로 굳이 금리인하에 손을 들지 않을 가능성도 있어 보이기 때문이다의사록을 보면 금리인하를 명백히 주장한 정해방 위원F위원은 선제적 대응 필요성을 강조하며 이달에도 인하에 손을 들 것으로 보인다 다만 한가지 짚어볼 점은 지난달 인하를 주장했던 그보다 동결에 손을 들었던 다른 일부 금통위원들에게서 경제를 보는 고민이 더 깊었다는 점이다 금통위 의사록을 보면 그는 세월호 사고의 영향이 일시적이고 제한적인 것인지 또는 장기적인 것인지를 예단하기는 어렵다며 최근 상황을 종합해 볼 때 경제주체들의 심리 위축이 장기화될 것으로 우려됨에 따라 현시점에서의 선제적인 경기대응이 필요한 것으로 판단된다고 밝혔을 뿐이다 14개월만에 나온 금리 인하 논거로는 왠지 두루뭉술한 측면이 커 그가 단지 정책조화 측면에서 인하에 손을 들었다고 밖에는 해석할 수 없는 대목이다 그렇잖아도 그는 기획예산처 차관 출신 인사인데다 기획재정부 추천 금통위원이다하성근 위원으로 추정되는 D위원도 이달 인하로 돌아설 것으로 보인다 그는 지난달 대부분의 시장참가자들이 이번 달 기준금리 동결을 기대하고 있다는 점도 고려해야 할 것이라고 밝혔다 이는 지난달 금리인하를 시사하는 듯한 논문발와 금통위 직전 한 세미나에 참석하면서 불러일으킨 논란 그리고 소수의견이 나온다면 하 위원일 것이라는 시장의 압도적 전망 등이 그를 위축시켰던 결과로밖에 해석되지 않는다 반대로 이달 기준금리 인하 기대감이 크다는 점을 고려해볼 때 하 위원은 부담없이 인하로 돌아설 것으로 보인다반면 문우식 위원으로 예상하는 E위원은 지난달에 이어 이달에도 동결 기조를 유지할 것으로 전망한다 그는 지난달 금통위에서 세월호 사고는 일시적인 충격으로 국내경제의 성장경로를 바꿀 정도는 아닌 것으로 판단된다며 일시적인 충격과 퍼머넌트 쇼크permanent shock를 구분할 필요가 있다고 밝힌 바 있다금통위원 사실상 3대3 갈릴듯이 총재 의지 따라 6대1 인하 or 4대3 동결문제는 장병화 정순원 함준호 위원이다 공개된 지난달 의사록만 봤을 때 다분히 중립에 가까운 비둘기로 보이기 때문이다우선 장병화 위원부총재으로 추정되는 C위원의 경우 성장경로나 GDP갭 축소 추세가 아직 꺾이지 않았다는 점 정책여력 확보를 강조하고 있다는 점에서 동결 가능성에 무게를 둔다 그렇잖아도 최근 전쟁이라는 글로벌 지정학적 리스크가 커지며 경제에 어떤 영향을 미칠지 모르는 상황이 전개되고 있다C위원을 장 부총재로 예상하는 이유는 이주열 총재가 지난달 금통위까지만 해도 금리인하 가능성에 상당한 거부감을 한 것으로 추정되기 때문이다 이 총재는 지난달 10일 금통위 기자회견에서 마이너스 GDP갭 성장세 둔화등 사실상 인하를 강력히 시사하는 문구들이 통화정책방향에 삽입된 것과 관련해 금리조정의 과거사례를 쭉 언급했는데 그런 과거사례를 염두에 둔 것은 전혀 아니다며 강하게 반발한 바 있다B위원은 금융중개지원대출에 만족한다면 동결에 손을 들겠지만 다양한 정책수단을 유연하게 발휘해야 한다는 점에서 금리인하도 염두했을수 있다 A위원은 최근 원화절상 압력이 급격히 줄었다는 점에서 일부 우려가 줄었을 수도 있겠다 하지만 세계경제 하방위험 저물가 지속 등은 여전히 인하에 무게를 둘 수 있는 변수다 다만 그는 지난달 비교적 담담하게 현 경제상황을 짚었었다결국 이들 세 위원의 판단은 이 총재의 결단에 크게 좌우될 수밖에 없어 보인다 그나마 동결 가능성이 있는 C위원이 장 부총재가 맞는다면 이 총재가 인하로 돌아설 경우 같은 집행부라는 점에서 인하로 돌아설 가능성이 높다 다만 그 역시 통화정책 전문가라는 점에서 꼭 총재와 세트로 판단하는 것은 잘못된 판단일수도 있음을 밝혀둔다어쨌든 이 총재가 인하로 돌아설 경우 이들 세 위원 역시 인하에 동참할 가능성이 높다 결국 6대1 인하가 이뤄질 것이다 반면 이 총재가 동결을 고수한다면 C위원으로 추정되는 부총재는 물론 A내지 B위원도 동결에 손을 들 것으로 보인다 이 경우 4대3 동결 가능성도 배제할 수 없겠다,https://www.edaily.co.kr/News/Read?newsId=01095526606186336&mediaCodeNo=257
2,2014.08.11,집 사라고 대출규제 풀었지만…집 안사고 생활비로 쓴다,정부가 주택시장 활성화를 위해 LTVDTI와 같은 대출규제를 풀었지만 정작 은행에선 정부의 정책 취지와는 다른 모습이 연출되고 있다 새로 은행 대출을 받아 집을 사겠다는 사람들의 발길은 뜸하지만 오히려 집을 담보로 대학 다니는 자식의 학자금을 마련하는 등 생활자금을 빌리려는 수요가 더 몰리고 있어서다 주택경기가 좀처럼 살아날 기미를 보이지 않다 보니 정부의 예상과 달리 주택담보대출을 받아 집을 사려는 수요 역시 움직이지 않고 있다는 게 금융권의 분석이다 문제는 정부가 정책 목적으로 내세운 주택시장 활성화와는 상관없이 확대된 주택담보대출이 자영업자 사업자금 등 생활자금으로 쓰일 개연성이 높아졌다는 점이다 빚에 빚을 더하는 셈인데 결국엔 가계부채에 악영향을 초래할 수 있다는 지적이 나온다 최근 금융당국은 이런 점을 우려해 시중은행 여신 담당 임원들을 소집해 될 수 있으면 확대된 주택담보대출이 정책

In [7]:
df = df_droped_na

In [8]:
df.to_csv('News_ready2tokenize.csv', index=False, encoding='utf-8-sig')

## Tokenization (멀티스레드)

In [2]:
df = pd.read_csv(r'C:\Users\ran14\Workspace\BOK_workspace\preprocessing\News_ready2tokenize.csv')
pd.set_option('display.max_colwidth', None)
df

,Date,Title,Content,Link
0,2014.08.11,뉴욕환시주간 달러 지정학적 불안에 조정 압박,이번 주1115일 뉴욕 외환시장에서 달러화는 지정학적 불안으로 단기적으로 하락 압력을 받을 것으로 예상된다지난 8일 미국시간 달러는 지정학적 불안 고조로 유로화와 엔화에 하락했다 투자자들이 지정학적 이슈를 포지션 조정의 빌미로 삼은 때문이다유로달러는 뉴욕 후장에서 전장 대비 03 오른 13410달러 근처에서 거래됐고 달러엔은 01 떨어진 10204엔에서 거래됐다유로화는 과도한 달러 롱포지션 청산에 따른 숏커버링으로 달러에 대해 올랐다 또 러시아와 우크라이나를 둘러싼 지정학적 긴장이 다소 완화된 점도 유로화 반등에 일조했다 우크라이나 국경 근처에서 진행된 러시아의 군사훈련이 예정대로 끝났다는 소식에 불안이 완화됐다그러나 지정학적 불안은 당분간 시장 참가자들이 주목할 이슈가 될 전망이다 미국이 이라크에 대한 선별적 공습을 개시한 데다 가자지구를 둘러싼 불안도 지속되고 있어 투자자들이 지정학적 불안을 포지션 조정의 빌미로 삼을 수 있기 때문이다버락 오바마 미국 대통령은 이라크 사태와 관련해 수주 안에 이 문제를 해결할 수 있을 것으로 보지 않으며 앞으로 시간이 좀 걸릴 것이라고 언급해 이라크 개입 장기화 가능성을 예고했다씨티그룹의 스티븐 잉글랜더 주요 10개국G10 외환 전략 헤드는 금융 매니저들이 고객들에게 구축한 포지션에서 나올 것을 권고해 지난 8일 달러 매도유로 매수 움직임이 일었다고 설명했다웨스턴 유니언의 조 마님보 선임 시장 애널리스트는 달러 강세가 차익 실현에 자리를 내줬다고 말했다그동안 투자자들이 달러화에 대해 과도하게 많은 롱포지션을 구축해온 때문이다달러지수는 지난 6일에는 81714까지 오르며 4주 연속 상승세로 마감했다 지수는 최근 81395까지 하락했으나 여전히 9개월래 최고치 근처에서 움직이고 있다 애널리스트들은 지정학적 불안에 따른 포지션 조정이 단기에 그칠 것으로 예상했다웨스트팩 뱅킹의 애널리스트들은 이번 지정학적 불안이 초기에 위험회피 심리를 일으켰지만 하루 이틀만에 시장은 이를 넘겨버렸다고 말했다이에 따라 이번 주 시장 참가자들은 3분기 국내총생산GDP을 가늠할 7월 소매판매 지를 주목할 것으로 예상된다 달러화가 강세를 재개하려면 미국 연방준비제도Fed의 조기 금리 인상 신호가 나타나야 하기 때문이다 그러나 마님보 애널리스트는 시장에 연방준비제도Fed가 금리를 올리기까지 오랜 시간이 걸릴 것이라는 인식이 있다고 전했다,https://news.einfomax.co.kr/news/articleView.html?idxno=116756
1,2014.08.11,8월 금통위 박빙 동결도 가능이주열 결단이 가른다③,한국은행 7월 금융통화위원회 의사록을 통해 나타난 6명의 금통위원이 본 경기판단은 전반적으로 비둘기완화적이라는데 이견이 없다 다만 비둘기적이었다고 해서 바로 인하로 돌아설 가능성은 낮아 보인다 일부 위원들 사이에서 최근 경제상황 변화와 중간에 한은이 단행한 금융중개지원대출 3조원 확대 등으로 굳이 금리인하에 손을 들지 않을 가능성도 있어 보이기 때문이다의사록을 보면 금리인하를 명백히 주장한 정해방 위원F위원은 선제적 대응 필요성을 강조하며 이달에도 인하에 손을 들 것으로 보인다 다만 한가지 짚어볼 점은 지난달 인하를 주장했던 그보다 동결에 손을 들었던 다른 일부 금통위원들에게서 경제를 보는 고민이 더 깊었다는 점이다 금통위 의사록을 보면 그는 세월호 사고의 영향이 일시적이고 제한적인 것인지 또는 장기적인 것인지를 예단하기는 어렵다며 최근 상황을 종합해 볼 때 경제주체들의 심리 위축이 장기화될 것으로 우려됨에 따라 현시점에서의 선제적인 경기대응이 필요한 것으로 판단된다고 밝혔을 뿐이다 14개월만에 나온 금리 인하 논거로는 왠지 두루뭉술한 측면이 커 그가 단지 정책조화 측면에서 인하에 손을 들었다고 밖에는 해석할 수 없는 대목이다 그렇잖아도 그는 기획예산처 차관 출신 인사인데다 기획재정부 추천 금통위원이다하성근 위원으로 추정되는 D위원도 이달 인하로 돌아설 것으로 보인다 그는 지난달 대부분의 시장참가자들이 이번 달 기준금리 동결을 기대하고 있다는 점도 고려해야 할 것이라고 밝혔다 이는 지난달 금리인하를 시사하는 듯한 논문발와 금통위 직전 한 세미나에 참석하면서 불러일으킨 논란 그리고 소수의견이 나온다면 하 위원일 것이라는 시장의 압도적 전망 등이 그를 위축시켰던 결과로밖에 해석되지 않는다 반대로 이달 기준금리 인하 기대감이 크다는 점을 고려해볼 때 하 위원은 부담없이 인하로 돌아설 것으로 보인다반면 문우식 위원으로 예상하는 E위원은 지난달에 이어 이달에도 동결 기조를 유지할 것으로 전망한다 그는 지난달 금통위에서 세월호 사고는 일시적인 충격으로 국내경제의 성장경로를 바꿀 정도는 아닌 것으로 판단된다며 일시적인 충격과 퍼머넌트 쇼크permanent shock를 구분할 필요가 있다고 밝힌 바 있다금통위원 사실상 3대3 갈릴듯이 총재 의지 따라 6대1 인하 or 4대3 동결문제는 장병화 정순원 함준호 위원이다 공개된 지난달 의사록만 봤을 때 다분히 중립에 가까운 비둘기로 보이기 때문이다우선 장병화 위원부총재으로 추정되는 C위원의 경우 성장경로나 GDP갭 축소 추세가 아직 꺾이지 않았다는 점 정책여력 확보를 강조하고 있다는 점에서 동결 가능성에 무게를 둔다 그렇잖아도 최근 전쟁이라는 글로벌 지정학적 리스크가 커지며 경제에 어떤 영향을 미칠지 모르는 상황이 전개되고 있다C위원을 장 부총재로 예상하는 이유는 이주열 총재가 지난달 금통위까지만 해도 금리인하 가능성에 상당한 거부감을 한 것으로 추정되기 때문이다 이 총재는 지난달 10일 금통위 기자회견에서 마이너스 GDP갭 성장세 둔화등 사실상 인하를 강력히 시사하는 문구들이 통화정책방향에 삽입된 것과 관련해 금리조정의 과거사례를 쭉 언급했는데 그런 과거사례를 염두에 둔 것은 전혀 아니다며 강하게 반발한 바 있다B위원은 금융중개지원대출에 만족한다면 동결에 손을 들겠지만 다양한 정책수단을 유연하게 발휘해야 한다는 점에서 금리인하도 염두했을수 있다 A위원은 최근 원화절상 압력이 급격히 줄었다는 점에서 일부 우려가 줄었을 수도 있겠다 하지만 세계경제 하방위험 저물가 지속 등은 여전히 인하에 무게를 둘 수 있는 변수다 다만 그는 지난달 비교적 담담하게 현 경제상황을 짚었었다결국 이들 세 위원의 판단은 이 총재의 결단에 크게 좌우될 수밖에 없어 보인다 그나마 동결 가능성이 있는 C위원이 장 부총재가 맞는다면 이 총재가 인하로 돌아설 경우 같은 집행부라는 점에서 인하로 돌아설 가능성이 높다 다만 그 역시 통화정책 전문가라는 점에서 꼭 총재와 세트로 판단하는 것은 잘못된 판단일수도 있음을 밝혀둔다어쨌든 이 총재가 인하로 돌아설 경우 이들 세 위원 역시 인하에 동참할 가능성이 높다 결국 6대1 인하가 이뤄질 것이다 반면 이 총재가 동결을 고수한다면 C위원으로 추정되는 부총재는 물론 A내지 B위원도 동결에 손을 들 것으로 보인다 이 경우 4대3 동결 가능성도 배제할 수 없겠다,https://www.edaily.co.kr/News/Read?newsId=01095526606186336&mediaCodeNo=257
2,2014.08.11,집 사라고 대출규제 풀었지만…집 안사고 생활비로 쓴다,정부가 주택시장 활성화를 위해 LTVDTI와 같은 대출규제를 풀었지만 정작 은행에선 정부의 정책 취지와는 다른 모습이 연출되고 있다 새로 은행 대출을 받아 집을 사겠다는 사람들의 발길은 뜸하지만 오히려 집을 담보로 대학 다니는 자식의 학자금을 마련하는 등 생활자금을 빌리려는 수요가 더 몰리고 있어서다 주택경기가 좀처럼 살아날 기미를 보이지 않다 보니 정부의 예상과 달리 주택담보대출을 받아 집을 사려는 수요 역시 움직이지 않고 있다는 게 금융권의 분석이다 문제는 정부가 정책 목적으로 내세운 주택시장 활성화와는 상관없이 확대된 주택담보대출이 자영업자 사업자금 등 생활자금으로 쓰일 개연성이 높아졌다는 점이다 빚에 빚을 더하는 셈인데 결국엔 가계부채에 악영향을 초래할 수 있다는 지적이 나온다 최근 금융당국은 이런 점을 우려해 시중은행 여신 담당 임원들을 소집해 될 수 있으면 확대된 주택담보대출이 정책

In [5]:
# 결측치 확인
print(df.isnull().sum())
count_NaN = df[df.isnull().any(axis=1)]
count_NaN

Date       0
Title      0
Content    0
Link       0
dtype: int64


,Date,Title,Content,Link


In [7]:
import pandas as pd
from ekonlpy.tag import Mecab

In [8]:
mecab = Mecab()

In [8]:
News = pd.read_csv(r'C:\Users\ran14\Workspace\BOK_workspace\preprocessing\News_ready2tokenize.csv')
News_content = News['Content']
News_tilte = News['Title']

tokenizer = Mecab()

def tokenize_text(text):
    return tokenizer.morphs(text)

News['Tokenized_Title'] = News_tilte.apply(tokenize_text)
News['Tokenized_Content'] = News_content.apply(tokenize_text)

content_tokens = News.Tokenized_Content
title_tokens = News.Tokenized_Title

KeyboardInterrupt: 

In [7]:
# 각 열에서 NoneType 데이터의 개수 확인
none_counts = df.isnull().sum()
print(none_counts)

Date       0
Title      0
Content    0
Link       0
dtype: int64


In [11]:
import concurrent.futures

# CSV 파일 읽기
News = pd.read_csv(r'C:\Users\ran14\Workspace\BOK_workspace\preprocessing\News_ready2tokenize.csv')
News_content = News['Content']
News_tilte = News['Title']

# MeCab 토크나이저 초기화
tokenizer = Mecab()

# 토크나이징 함수 정의
def tokenize_text(text):
    return tokenizer.morphs(text)

# 멀티스레딩을 사용한 병렬 처리 함수 정의
def parallel_apply(series, func, n_threads=4):
    with concurrent.futures.ThreadPoolExecutor(max_workers=n_threads) as executor:
        results = list(executor.map(func, series))
    return results

# 멀티스레드를 사용하여 토크나이징 적용
News['Tokenized_Title'] = parallel_apply(News_tilte, tokenize_text)
News['Tokenized_Content'] = parallel_apply(News_content, tokenize_text)

content_tokens = News['Tokenized_Content']
title_tokens = News['Tokenized_Title']

In [31]:
df.to_csv('edaily_tokenization_01', encoding='utf-8-sig')

## NoneType 오류 발생

- 결측치 없음 확인
- 멀티스레드 오류 추정 (멀티스레드 적용 안 한 경우 정상 동작)
- 데이터 축소 위해 라벨링 과정에서 확인 된 중립 데이터, 휴일에 해당하는 기사 제거

In [20]:
label = pd.read_csv(r'C:\Users\ran14\Workspace\BOK_workspace\preprocessing\News_UL_labeled_4.csv')
pd.set_option('display.max_colwidth', None)
label

,News Date,Later Date,Current Rate,Later Rate,Difference,Label
0,2014-08-13,2014-10-13,2.49,2.23,-0.26,0
1,2014-08-13,2014-10-13,2.49,2.23,-0.26,0
2,2014-08-13,2014-10-13,2.49,2.23,-0.26,0
3,2014-08-13,2014-10-13,2.49,2.23,-0.26,0
4,2014-08-13,2014-10-13,2.49,2.23,-0.26,0
...,...,...,...,...,...,...
129155,2024-06-07,2024-08-07,3.50,3.53,0.03,1
129156,2024-06-07,2024-08-07,3.50,3.53,0.03,1
129157,2024-06-07,2024-08-07,3.50,3.53,0.03,1
129158,2024-06-07,2024-08-07,3.50,3.53,0.03,1


In [21]:
label_filtered = label[label['Label'] != 'N']
print(label_filtered)

         News Date  Later Date  Current Rate  Later Rate  Difference Label
0       2014-08-13  2014-10-13          2.49        2.23       -0.26     0
1       2014-08-13  2014-10-13          2.49        2.23       -0.26     0
2       2014-08-13  2014-10-13          2.49        2.23       -0.26     0
3       2014-08-13  2014-10-13          2.49        2.23       -0.26     0
4       2014-08-13  2014-10-13          2.49        2.23       -0.26     0
...            ...         ...           ...         ...         ...   ...
129155  2024-06-07  2024-08-07          3.50        3.53        0.03     1
129156  2024-06-07  2024-08-07          3.50        3.53        0.03     1
129157  2024-06-07  2024-08-07          3.50        3.53        0.03     1
129158  2024-06-07  2024-08-07          3.50        3.53        0.03     1
129159  2024-06-07  2024-08-07          3.50        3.53        0.03     1

[87912 rows x 6 columns]


In [22]:
# Date 고유 값 확인
label_unique = label_filtered['News Date'].unique()
len(label_unique)

1007

In [23]:
# 중복 Date 삭제 데이터프레임 확인
label_unique_df = label_filtered.drop_duplicates(subset=['News Date'])
print(label_unique_df)

         News Date  Later Date  Current Rate  Later Rate  Difference Label
0       2014-08-13  2014-10-13          2.49        2.23       -0.26     0
65      2014-08-14  2014-10-14          2.49        2.23       -0.26     0
198     2014-08-20  2014-10-20          2.24        1.98       -0.26     0
247     2014-08-21  2014-10-21          2.23        1.98       -0.25     0
353     2014-08-22  2014-10-22          2.23        1.98       -0.25     0
...            ...         ...           ...         ...         ...   ...
128562  2024-05-23  2024-07-23          3.53        3.51       -0.02     0
128769  2024-05-29  2024-07-29          3.52        3.54        0.02     1
128863  2024-05-30  2024-07-30          3.52        3.50       -0.02     0
129044  2024-06-05  2024-08-05          3.60        3.54       -0.06     0
129103  2024-06-07  2024-08-07          3.50        3.53        0.03     1

[1007 rows x 6 columns]


### 중립 및 휴일이 아닌 기사로 데이터 축소

In [28]:
import pandas as pd

In [31]:
# 시리즈로 변경
label_unique = pd.Series(label_unique)
# label_unique 날짜 포맷 변경
label_unique = label_unique.str.replace('-', '.')

In [34]:
df = pd.read_csv(r'C:\Users\ran14\Workspace\BOK_workspace\preprocessing\News_ready2tokenize.csv')
pd.set_option('display.max_colwidth', None)
df

,Date,Title,Content,Link
0,2014.08.11,뉴욕환시주간 달러 지정학적 불안에 조정 압박,이번 주1115일 뉴욕 외환시장에서 달러화는 지정학적 불안으로 단기적으로 하락 압력을 받을 것으로 예상된다지난 8일 미국시간 달러는 지정학적 불안 고조로 유로화와 엔화에 하락했다 투자자들이 지정학적 이슈를 포지션 조정의 빌미로 삼은 때문이다유로달러는 뉴욕 후장에서 전장 대비 03 오른 13410달러 근처에서 거래됐고 달러엔은 01 떨어진 10204엔에서 거래됐다유로화는 과도한 달러 롱포지션 청산에 따른 숏커버링으로 달러에 대해 올랐다 또 러시아와 우크라이나를 둘러싼 지정학적 긴장이 다소 완화된 점도 유로화 반등에 일조했다 우크라이나 국경 근처에서 진행된 러시아의 군사훈련이 예정대로 끝났다는 소식에 불안이 완화됐다그러나 지정학적 불안은 당분간 시장 참가자들이 주목할 이슈가 될 전망이다 미국이 이라크에 대한 선별적 공습을 개시한 데다 가자지구를 둘러싼 불안도 지속되고 있어 투자자들이 지정학적 불안을 포지션 조정의 빌미로 삼을 수 있기 때문이다버락 오바마 미국 대통령은 이라크 사태와 관련해 수주 안에 이 문제를 해결할 수 있을 것으로 보지 않으며 앞으로 시간이 좀 걸릴 것이라고 언급해 이라크 개입 장기화 가능성을 예고했다씨티그룹의 스티븐 잉글랜더 주요 10개국G10 외환 전략 헤드는 금융 매니저들이 고객들에게 구축한 포지션에서 나올 것을 권고해 지난 8일 달러 매도유로 매수 움직임이 일었다고 설명했다웨스턴 유니언의 조 마님보 선임 시장 애널리스트는 달러 강세가 차익 실현에 자리를 내줬다고 말했다그동안 투자자들이 달러화에 대해 과도하게 많은 롱포지션을 구축해온 때문이다달러지수는 지난 6일에는 81714까지 오르며 4주 연속 상승세로 마감했다 지수는 최근 81395까지 하락했으나 여전히 9개월래 최고치 근처에서 움직이고 있다 애널리스트들은 지정학적 불안에 따른 포지션 조정이 단기에 그칠 것으로 예상했다웨스트팩 뱅킹의 애널리스트들은 이번 지정학적 불안이 초기에 위험회피 심리를 일으켰지만 하루 이틀만에 시장은 이를 넘겨버렸다고 말했다이에 따라 이번 주 시장 참가자들은 3분기 국내총생산GDP을 가늠할 7월 소매판매 지를 주목할 것으로 예상된다 달러화가 강세를 재개하려면 미국 연방준비제도Fed의 조기 금리 인상 신호가 나타나야 하기 때문이다 그러나 마님보 애널리스트는 시장에 연방준비제도Fed가 금리를 올리기까지 오랜 시간이 걸릴 것이라는 인식이 있다고 전했다,https://news.einfomax.co.kr/news/articleView.html?idxno=116756
1,2014.08.11,8월 금통위 박빙 동결도 가능이주열 결단이 가른다③,한국은행 7월 금융통화위원회 의사록을 통해 나타난 6명의 금통위원이 본 경기판단은 전반적으로 비둘기완화적이라는데 이견이 없다 다만 비둘기적이었다고 해서 바로 인하로 돌아설 가능성은 낮아 보인다 일부 위원들 사이에서 최근 경제상황 변화와 중간에 한은이 단행한 금융중개지원대출 3조원 확대 등으로 굳이 금리인하에 손을 들지 않을 가능성도 있어 보이기 때문이다의사록을 보면 금리인하를 명백히 주장한 정해방 위원F위원은 선제적 대응 필요성을 강조하며 이달에도 인하에 손을 들 것으로 보인다 다만 한가지 짚어볼 점은 지난달 인하를 주장했던 그보다 동결에 손을 들었던 다른 일부 금통위원들에게서 경제를 보는 고민이 더 깊었다는 점이다 금통위 의사록을 보면 그는 세월호 사고의 영향이 일시적이고 제한적인 것인지 또는 장기적인 것인지를 예단하기는 어렵다며 최근 상황을 종합해 볼 때 경제주체들의 심리 위축이 장기화될 것으로 우려됨에 따라 현시점에서의 선제적인 경기대응이 필요한 것으로 판단된다고 밝혔을 뿐이다 14개월만에 나온 금리 인하 논거로는 왠지 두루뭉술한 측면이 커 그가 단지 정책조화 측면에서 인하에 손을 들었다고 밖에는 해석할 수 없는 대목이다 그렇잖아도 그는 기획예산처 차관 출신 인사인데다 기획재정부 추천 금통위원이다하성근 위원으로 추정되는 D위원도 이달 인하로 돌아설 것으로 보인다 그는 지난달 대부분의 시장참가자들이 이번 달 기준금리 동결을 기대하고 있다는 점도 고려해야 할 것이라고 밝혔다 이는 지난달 금리인하를 시사하는 듯한 논문발와 금통위 직전 한 세미나에 참석하면서 불러일으킨 논란 그리고 소수의견이 나온다면 하 위원일 것이라는 시장의 압도적 전망 등이 그를 위축시켰던 결과로밖에 해석되지 않는다 반대로 이달 기준금리 인하 기대감이 크다는 점을 고려해볼 때 하 위원은 부담없이 인하로 돌아설 것으로 보인다반면 문우식 위원으로 예상하는 E위원은 지난달에 이어 이달에도 동결 기조를 유지할 것으로 전망한다 그는 지난달 금통위에서 세월호 사고는 일시적인 충격으로 국내경제의 성장경로를 바꿀 정도는 아닌 것으로 판단된다며 일시적인 충격과 퍼머넌트 쇼크permanent shock를 구분할 필요가 있다고 밝힌 바 있다금통위원 사실상 3대3 갈릴듯이 총재 의지 따라 6대1 인하 or 4대3 동결문제는 장병화 정순원 함준호 위원이다 공개된 지난달 의사록만 봤을 때 다분히 중립에 가까운 비둘기로 보이기 때문이다우선 장병화 위원부총재으로 추정되는 C위원의 경우 성장경로나 GDP갭 축소 추세가 아직 꺾이지 않았다는 점 정책여력 확보를 강조하고 있다는 점에서 동결 가능성에 무게를 둔다 그렇잖아도 최근 전쟁이라는 글로벌 지정학적 리스크가 커지며 경제에 어떤 영향을 미칠지 모르는 상황이 전개되고 있다C위원을 장 부총재로 예상하는 이유는 이주열 총재가 지난달 금통위까지만 해도 금리인하 가능성에 상당한 거부감을 한 것으로 추정되기 때문이다 이 총재는 지난달 10일 금통위 기자회견에서 마이너스 GDP갭 성장세 둔화등 사실상 인하를 강력히 시사하는 문구들이 통화정책방향에 삽입된 것과 관련해 금리조정의 과거사례를 쭉 언급했는데 그런 과거사례를 염두에 둔 것은 전혀 아니다며 강하게 반발한 바 있다B위원은 금융중개지원대출에 만족한다면 동결에 손을 들겠지만 다양한 정책수단을 유연하게 발휘해야 한다는 점에서 금리인하도 염두했을수 있다 A위원은 최근 원화절상 압력이 급격히 줄었다는 점에서 일부 우려가 줄었을 수도 있겠다 하지만 세계경제 하방위험 저물가 지속 등은 여전히 인하에 무게를 둘 수 있는 변수다 다만 그는 지난달 비교적 담담하게 현 경제상황을 짚었었다결국 이들 세 위원의 판단은 이 총재의 결단에 크게 좌우될 수밖에 없어 보인다 그나마 동결 가능성이 있는 C위원이 장 부총재가 맞는다면 이 총재가 인하로 돌아설 경우 같은 집행부라는 점에서 인하로 돌아설 가능성이 높다 다만 그 역시 통화정책 전문가라는 점에서 꼭 총재와 세트로 판단하는 것은 잘못된 판단일수도 있음을 밝혀둔다어쨌든 이 총재가 인하로 돌아설 경우 이들 세 위원 역시 인하에 동참할 가능성이 높다 결국 6대1 인하가 이뤄질 것이다 반면 이 총재가 동결을 고수한다면 C위원으로 추정되는 부총재는 물론 A내지 B위원도 동결에 손을 들 것으로 보인다 이 경우 4대3 동결 가능성도 배제할 수 없겠다,https://www.edaily.co.kr/News/Read?newsId=01095526606186336&mediaCodeNo=257
2,2014.08.11,집 사라고 대출규제 풀었지만…집 안사고 생활비로 쓴다,정부가 주택시장 활성화를 위해 LTVDTI와 같은 대출규제를 풀었지만 정작 은행에선 정부의 정책 취지와는 다른 모습이 연출되고 있다 새로 은행 대출을 받아 집을 사겠다는 사람들의 발길은 뜸하지만 오히려 집을 담보로 대학 다니는 자식의 학자금을 마련하는 등 생활자금을 빌리려는 수요가 더 몰리고 있어서다 주택경기가 좀처럼 살아날 기미를 보이지 않다 보니 정부의 예상과 달리 주택담보대출을 받아 집을 사려는 수요 역시 움직이지 않고 있다는 게 금융권의 분석이다 문제는 정부가 정책 목적으로 내세운 주택시장 활성화와는 상관없이 확대된 주택담보대출이 자영업자 사업자금 등 생활자금으로 쓰일 개연성이 높아졌다는 점이다 빚에 빚을 더하는 셈인데 결국엔 가계부채에 악영향을 초래할 수 있다는 지적이 나온다 최근 금융당국은 이런 점을 우려해 시중은행 여신 담당 임원들을 소집해 될 수 있으면 확대된 주택담보대출이 정책

In [35]:
df_unique = df[df['Date'].isin(label_unique)]
df_unique

,Date,Title,Content,Link
147,2014.08.13,관심병사 베테랑 되기,자고 일어나기 무섭게 관심병사들이 스스로 목숨을 끊는 사건이 연이어 터지고 있다 지난 12일 육군 28사단 소속 관심병사 2명이 휴가를 나와 동반자살했고 제3군 사령부 직할부대 관심병사도 자신의 총으로 스스로 목숨을 끊어 주위를 안타깝게 하고 있다 저승에서나마 이승에서 누리지 못한 자유를 누릴 수 있기를 바란다증권시장에서 증권주는 관심병사와 같은 존재였다 박스권이 갖힌 증시에서 먼저 노란색 견장을 단 조선 해운 건설주들 만큼이나 앞이 보이지 않기는 마찬가지였다 연이은 적자 행진에 대규모 구조조정 소식으로 신문지상을 도배할 뿐 별다른 기쁜 소식을 찾기 어려운 증권업종을 보는 시선은 안타까움이었다 증권사들도 관심병사들 만큼이나 자유가 없었던 탓일 것이다 정부가 증시를 억누르던 규제를 풀어 자유를 주자 전일12일 증권업종 주가는 날개를 달았다정부는 증시 가격제한폭을 단계적으로 현재의 두배인 상하 30까지 확대하겠다는 내용을 투자활성화대책에 포함시켰다 더불어 최경환 경제팀이 내수경기 부양을 천명하고 배당확대를 유도하는 대책들을 내놓으면서 증권주는 노란색 견장을 떼고 분대를 이끄는 베테랑 병사에게 주어지는 녹색 견장으로 바꿔달 기세다 증시를 이끄는 원톱이 될수 있겠다는 기대감까지 감돌고 있다증시 전문가들은 상하한가 가격제한폭을 확대하는 정책이 실제 시행되면 박스권에 갇힌 증시에도 큰 도움이 되리란 의견을 내놓는다 이런 기대감 덕분인지 전일 코스피도 우크라이나와 이라크 등 지정학적 우려 속에서도 이틀 연속 상승세를 탔다 금리를 내릴 것으로 예상하는 이달 14일 금융통화위원회의 결과가 오늘 증시에도 단비가 될지 관심이 쏠린다증권주 주가가 오른 것은 더 폭넓어진 선택의 자유와 정부의 관심 덕분이다 주식 시장 관계자들에게 자신이 매매하고 싶은 가격을 스스로 선택할 자유 기업이 쌓아놓은 유보금을 투자자나 근로자들에게 나눠주도록 유도하는 정부의 관심 이 두 가지 덕분에 증권업종 주가에도 모처럼 훈풍이 부는 것이다 여전히 현대증권의 희망퇴직 소식으로 우울한 나날은 계속되고 있지만 업종을 보는 시장의 시선이 달라지고 있는 것은 분명해 보인다관심병사가 배테랑 병사가 되는 것도 마찬가지일 것이다 다양한 임무 중에서 굳이 적성에 맞지 않는 임무를 관심병사에게 줄 필요가 없다 그들에게 선택할 자유를 주고 관심병사라는 호칭에 걸맞게 선임병사와 장교들이 세심한 관심을 기울인다면 증권주 주가처럼 그들도 자신감을 찾을 수 있을 것이다,https://www.edaily.co.kr/News/Read?newsId=01272646606186992&mediaCodeNo=257
148,2014.08.13,하나외환은행 PB프라이빗 뱅커통합브랜드 연내 출범하나,하나은행과 외환은행의 조기통합이 그룹 내 화두로 떠오른 가운데 가장 먼저 통합이 이뤄질 부문은 PB프라이빗 뱅커부문이 될 것으로 보인다 PB는 전문가 집단이라는 인식이 강해 다른 사업부문보다 통합에 대한 저항감이 덜한데다 두 은행의 시너지가 빨리 가시화될 수 있는 부문이다12일 하나금융지주에 따르면 하나금융은 올해 초 외환은행 인수 당시 하나은행 PB 사업본부와 외환은행 PB 통합을 목로 통합 브랜드 골드 클럽을 올해 출범시킬 예정이었다 다만 지난 6월 김정태 회장의 조기통합론 카드를 꺼내 들면서 애초 PB부문을 먼저 하나로 묶는다는 계획은 다소 차질을 빚고 있지만 그룹 내에선 PB부문 통합이 가장 단시간 내에 가장 효과를 낼 것이란 데는 공감대가 이뤄진 상황이다 올 상반기 현재 하나은행과 외환은행의 PB 관리자산은 각각 약 23조원 10조원 등이며 PB 인력은 각각 230명 103명 정도다 따라서 두 은행 PB부문이 통합하면 자산 33조원에 330명 이상의 대형 PB조직이 탄생하게 된다 지난 7월 하나은행과 외환은행의 조기통합 추진을 위한 결의문에서도 두 은행의 PB업무 외국환 경쟁력 등 강점 공유로 인한 시너지를 225억원으로 분석하기도 했다특히 하나은행 PB센터는 국내 최초로 지난 1995년 현대적 개념의 PB 비즈니스를 도입하는 등 업계에서 강자로 통한다 1997년 세무지원서비스 2005년 첫 해외PB파견 2006년 토탈 라이프케어 2011년 상속증여센터 등도 국내 최초로 도입했다 또 예치자산 1억원 이상의 VIP클럽 5억원 이상의 골드클럽 10억원 이상의 WM그룹으로 세분화 돼 있는데다 골드클럽과 WM그룹은 각각 센터수 15개 2개 PB인원도 51명 10명으로 은행권에서 강점을 지난 부문으로 평가 받고 있다 외환은행 PB 역시 외국매매와 해외 채널 등에서 강점이 있어 두 은행은 상호보완성 효과를 톡톡히 누릴 수 있다는 설명이다 외환은행 PB 부문은 5억원 이상의 WMC와 1억원 이상의 영업점 PB센터를 각각 10개 85개 운영 중인데 PB의 70 이상이 외국환관련 자격증을 가지고 있는데다 해외동포 등 비거주고객 조달 비중이 높고 해외교포 VIP 연계 가능성이 큰 장점이 있다금융권 관계자는 하나은행의 경우 국민은행 신한은행과 달리 10억원 이상의 고액자산가 비중이 높아 PB 1인당 생산성이 굉장히 높다며 외환은행의 기업 PB 고객에 대한 하나은행의 PB 서비스가 접목되면 다른 은행에 비해 확연히 높은 수익이 기대된다고 말했다 실제 두 은행의 PB부문이 통합이 조금씩 가시화되면서 은행권은 긴장하고 있다 저금리 기조가 장기화 되면서 은행들이 PB부문을 강화를 미래먹거리 중 하나로 강조하는 분위기 속에서 리테일과 외국환에 강점을 지닌데다 규모의 경제까지 갖춘 하나외환은행 통합 PB부문은 위협이 될 수 있기 때문이다한 시중은행 PB담당 임원은 현재 5억원 이상 자산가들을 영입하느라 각 은행이 WM센터를 특화하고 PB전용상품 개발에 매진하는 등 고액자산가 고객 확보전이 뜨겁다며 특히 하나은행이 10억원 이상 고객이 많아 외환은행 PB부문과 통합까지 이뤄지면 하나은행PB의 인기는 더욱 치솟을 것으로 보여 업계로선 긴장하지 않을 수 없다고 말했다,https://www.edaily.co.kr/News/Read?newsId=01141446606186992&mediaCodeNo=257
149,2014.08.13,돈 되는 앱테크,천연화장품 애호가인 주부 김씨는 아이소이 앱을 통해 40 할인 쿠폰을 받았다 평소 아이소이 제품을 즐겨 사용했던 그는 앱 다운만으로 쿠폰을 받아서 뿌듯하다며 큰 할인 혜택을 받을 것 같아 앞으로도 계속 앱 쇼핑을 활용할 생각이라고 말했다최근 기업들이 앞다퉈 모바일 앱 시장으로 뛰어들면서 앱 다운만으로 할인되는 쿠폰들이 넘쳐나고 있다 같은 상품이라도 앱을 통해 쇼핑을 하면 훨씬 더 저렴한 가격으로 할인을 받기도 한다 스마트한 앱 쇼핑족들이 할인 혜택을 톡톡 누리고 있는 것이다 특히 이들은 주로 아침 저녁 출퇴근 시간을 적극 활용하는 것으로 조사됐다이번 재테크의 여왕은 알아두면 편리하고 돈이 되는 모바일 앱테크다 늘 손에 쥐고 있는 앱 하나만으로 활용할 수 있는 팁들이 무궁무진하다 ①신규 앱다운 할인 이벤트 적극활용아침저녁 출퇴근 시간에 모바일 쇼핑을 즐기는 이씨는 신규 앱다운 할인 쿠폰 받기에 재미가 들렸다 주로 활용하는 모바일 앱은 물론이고 새롭게 이벤트가 진행되는 앱을 찾아 할인 쿠폰을 모으고 있다 롯데카드의 전자지갑 앱인 클러치를 다운받고 3000원 캐시백을 받았고 우리은행의 새로운 앱인 원클릭을 다운 받고 캘리포니아 피자의 1만원 상당의 샐러드 쿠폰을 받았다 이씨는 최근 신규 앱론칭 이벤트들이 많다며 이를 잘만 활용하면 재미가 쏠쏠하다고 말했다 ②모바일 앱결제 지갑 없어도 척척이씨가 또 즐겨 사용하는 것은 모바일 결제다 특히 세븐일레븐 등 편의점에선 선불형 교통카드인 캐시비 앱으로 결제한다 같은 제품이라도 캐시비로 결제를 하면 할인 혜택을 받을 수 있는데다 교통카드를 들고 다닐 필요가 없이 간편하기 때문이다 그는 쿠폰도 앱으로 모은다 주커피 등 일부 커피숍에서 모바일 앱 터칭을 통해 쿠폰을 적립하도록 하고 있다 예전처럼 지갑에 두툼하게 쿠폰들을 들고 다닐 필요가 없어진 셈이다 하지만 모바일 앱결제 기능 활용시 주의할 점은 NFC모드다 모바일 결제 때 필요한 NFC 기능을 평소에도 켜두면 휴대폰 배터리가 빨리 

In [38]:
df_filtered.to_csv('News_ready2tokenize_filtered.csv', encoding='utf-8-sig')

## 2nd Tokenization

In [2]:
df = pd.read_csv(r'C:\Users\ran14\Workspace\BOK_workspace\preprocessing\News_ready2tokenize_filtered.csv')
pd.set_option('display.max_colwidth', None)
df

,Unnamed: 0,Date,Title,Content,Link
0,147,2014.08.13,관심병사 베테랑 되기,자고 일어나기 무섭게 관심병사들이 스스로 목숨을 끊는 사건이 연이어 터지고 있다 지난 12일 육군 28사단 소속 관심병사 2명이 휴가를 나와 동반자살했고 제3군 사령부 직할부대 관심병사도 자신의 총으로 스스로 목숨을 끊어 주위를 안타깝게 하고 있다 저승에서나마 이승에서 누리지 못한 자유를 누릴 수 있기를 바란다증권시장에서 증권주는 관심병사와 같은 존재였다 박스권이 갖힌 증시에서 먼저 노란색 견장을 단 조선 해운 건설주들 만큼이나 앞이 보이지 않기는 마찬가지였다 연이은 적자 행진에 대규모 구조조정 소식으로 신문지상을 도배할 뿐 별다른 기쁜 소식을 찾기 어려운 증권업종을 보는 시선은 안타까움이었다 증권사들도 관심병사들 만큼이나 자유가 없었던 탓일 것이다 정부가 증시를 억누르던 규제를 풀어 자유를 주자 전일12일 증권업종 주가는 날개를 달았다정부는 증시 가격제한폭을 단계적으로 현재의 두배인 상하 30까지 확대하겠다는 내용을 투자활성화대책에 포함시켰다 더불어 최경환 경제팀이 내수경기 부양을 천명하고 배당확대를 유도하는 대책들을 내놓으면서 증권주는 노란색 견장을 떼고 분대를 이끄는 베테랑 병사에게 주어지는 녹색 견장으로 바꿔달 기세다 증시를 이끄는 원톱이 될수 있겠다는 기대감까지 감돌고 있다증시 전문가들은 상하한가 가격제한폭을 확대하는 정책이 실제 시행되면 박스권에 갇힌 증시에도 큰 도움이 되리란 의견을 내놓는다 이런 기대감 덕분인지 전일 코스피도 우크라이나와 이라크 등 지정학적 우려 속에서도 이틀 연속 상승세를 탔다 금리를 내릴 것으로 예상하는 이달 14일 금융통화위원회의 결과가 오늘 증시에도 단비가 될지 관심이 쏠린다증권주 주가가 오른 것은 더 폭넓어진 선택의 자유와 정부의 관심 덕분이다 주식 시장 관계자들에게 자신이 매매하고 싶은 가격을 스스로 선택할 자유 기업이 쌓아놓은 유보금을 투자자나 근로자들에게 나눠주도록 유도하는 정부의 관심 이 두 가지 덕분에 증권업종 주가에도 모처럼 훈풍이 부는 것이다 여전히 현대증권의 희망퇴직 소식으로 우울한 나날은 계속되고 있지만 업종을 보는 시장의 시선이 달라지고 있는 것은 분명해 보인다관심병사가 배테랑 병사가 되는 것도 마찬가지일 것이다 다양한 임무 중에서 굳이 적성에 맞지 않는 임무를 관심병사에게 줄 필요가 없다 그들에게 선택할 자유를 주고 관심병사라는 호칭에 걸맞게 선임병사와 장교들이 세심한 관심을 기울인다면 증권주 주가처럼 그들도 자신감을 찾을 수 있을 것이다,https://www.edaily.co.kr/News/Read?newsId=01272646606186992&mediaCodeNo=257
1,148,2014.08.13,하나외환은행 PB프라이빗 뱅커통합브랜드 연내 출범하나,하나은행과 외환은행의 조기통합이 그룹 내 화두로 떠오른 가운데 가장 먼저 통합이 이뤄질 부문은 PB프라이빗 뱅커부문이 될 것으로 보인다 PB는 전문가 집단이라는 인식이 강해 다른 사업부문보다 통합에 대한 저항감이 덜한데다 두 은행의 시너지가 빨리 가시화될 수 있는 부문이다12일 하나금융지주에 따르면 하나금융은 올해 초 외환은행 인수 당시 하나은행 PB 사업본부와 외환은행 PB 통합을 목로 통합 브랜드 골드 클럽을 올해 출범시킬 예정이었다 다만 지난 6월 김정태 회장의 조기통합론 카드를 꺼내 들면서 애초 PB부문을 먼저 하나로 묶는다는 계획은 다소 차질을 빚고 있지만 그룹 내에선 PB부문 통합이 가장 단시간 내에 가장 효과를 낼 것이란 데는 공감대가 이뤄진 상황이다 올 상반기 현재 하나은행과 외환은행의 PB 관리자산은 각각 약 23조원 10조원 등이며 PB 인력은 각각 230명 103명 정도다 따라서 두 은행 PB부문이 통합하면 자산 33조원에 330명 이상의 대형 PB조직이 탄생하게 된다 지난 7월 하나은행과 외환은행의 조기통합 추진을 위한 결의문에서도 두 은행의 PB업무 외국환 경쟁력 등 강점 공유로 인한 시너지를 225억원으로 분석하기도 했다특히 하나은행 PB센터는 국내 최초로 지난 1995년 현대적 개념의 PB 비즈니스를 도입하는 등 업계에서 강자로 통한다 1997년 세무지원서비스 2005년 첫 해외PB파견 2006년 토탈 라이프케어 2011년 상속증여센터 등도 국내 최초로 도입했다 또 예치자산 1억원 이상의 VIP클럽 5억원 이상의 골드클럽 10억원 이상의 WM그룹으로 세분화 돼 있는데다 골드클럽과 WM그룹은 각각 센터수 15개 2개 PB인원도 51명 10명으로 은행권에서 강점을 지난 부문으로 평가 받고 있다 외환은행 PB 역시 외국매매와 해외 채널 등에서 강점이 있어 두 은행은 상호보완성 효과를 톡톡히 누릴 수 있다는 설명이다 외환은행 PB 부문은 5억원 이상의 WMC와 1억원 이상의 영업점 PB센터를 각각 10개 85개 운영 중인데 PB의 70 이상이 외국환관련 자격증을 가지고 있는데다 해외동포 등 비거주고객 조달 비중이 높고 해외교포 VIP 연계 가능성이 큰 장점이 있다금융권 관계자는 하나은행의 경우 국민은행 신한은행과 달리 10억원 이상의 고액자산가 비중이 높아 PB 1인당 생산성이 굉장히 높다며 외환은행의 기업 PB 고객에 대한 하나은행의 PB 서비스가 접목되면 다른 은행에 비해 확연히 높은 수익이 기대된다고 말했다 실제 두 은행의 PB부문이 통합이 조금씩 가시화되면서 은행권은 긴장하고 있다 저금리 기조가 장기화 되면서 은행들이 PB부문을 강화를 미래먹거리 중 하나로 강조하는 분위기 속에서 리테일과 외국환에 강점을 지닌데다 규모의 경제까지 갖춘 하나외환은행 통합 PB부문은 위협이 될 수 있기 때문이다한 시중은행 PB담당 임원은 현재 5억원 이상 자산가들을 영입하느라 각 은행이 WM센터를 특화하고 PB전용상품 개발에 매진하는 등 고액자산가 고객 확보전이 뜨겁다며 특히 하나은행이 10억원 이상 고객이 많아 외환은행 PB부문과 통합까지 이뤄지면 하나은행PB의 인기는 더욱 치솟을 것으로 보여 업계로선 긴장하지 않을 수 없다고 말했다,https://www.edaily.co.kr/News/Read?newsId=01141446606186992&mediaCodeNo=257
2,149,2014.08.13,돈 되는 앱테크,천연화장품 애호가인 주부 김씨는 아이소이 앱을 통해 40 할인 쿠폰을 받았다 평소 아이소이 제품을 즐겨 사용했던 그는 앱 다운만으로 쿠폰을 받아서 뿌듯하다며 큰 할인 혜택을 받을 것 같아 앞으로도 계속 앱 쇼핑을 활용할 생각이라고 말했다최근 기업들이 앞다퉈 모바일 앱 시장으로 뛰어들면서 앱 다운만으로 할인되는 쿠폰들이 넘쳐나고 있다 같은 상품이라도 앱을 통해 쇼핑을 하면 훨씬 더 저렴한 가격으로 할인을 받기도 한다 스마트한 앱 쇼핑족들이 할인 혜택을 톡톡 누리고 있는 것이다 특히 이들은 주로 아침 저녁 출퇴근 시간을 적극 활용하는 것으로 조사됐다이번 재테크의 여왕은 알아두면 편리하고 돈이 되는 모바일 앱테크다 늘 손에 쥐고 있는 앱 하나만으로 활용할 수 있는 팁들이 무궁무진하다 ①신규 앱다운 할인 이벤트 적극활용아침저녁 출퇴근 시간에 모바일 쇼핑을 즐기는 이씨는 신규 앱다운 할인 쿠폰 받기에 재미가 들렸다 주로 활용하는 모바일 앱은 물론이고 새롭게 이벤트가 진행되는 앱을 찾아 할인 쿠폰을 모으고 있다 롯데카드의 전자지갑 앱인 클러치를 다운받고 3000원 캐시백을 받았고 우리은행의 새로운 앱인 원클릭을 다운 받고 캘리포니아 피자의 1만원 상당의 샐러드 쿠폰을 받았다 이씨는 최근 신규 앱론칭 이벤트들이 많다며 이를 잘만 활용하면 재미가 쏠쏠하다고 말했다 ②모바일 앱결제 지갑 없어도 척척이씨가 또 즐겨 사용하는 것은 모바일 결제다 특히 세븐일레븐 등 편의점에선 선불형 교통카드인 캐시비 앱으로 결제한다 같은 제품이라도 캐시비로 결제를 하면 할인 혜택을 받을 수 있는데다 교통카드를 들고 다닐 필요가 없이 간편하기 때문이다 그는 쿠폰도 앱으로 모은다 주커피 등 일부 커피숍에서 모바일 앱 터칭을 통해 쿠폰을 적립하도록 하고 있다 예전처럼 지갑에 두툼하게 쿠폰들을 들고 다닐 필요가 없어진 셈이다 하지만 모바일 앱결제 기능 활용시 주의할 점은 NFC모드다 모바일 결제 때 필요한 NFC 기능을 평소에도

In [3]:
from ekonlpy.tag import Mecab

tokenizer = Mecab()

In [4]:
def tokenize_text(text):
    return tokenizer.morphs(text)

df['Title_Tokens'] = df['Title'].apply(tokenize_text)
df['Content_Tokens'] = df['Content'].apply(tokenize_text)

df

,Unnamed: 0,Date,Title,Content,Link,Title_Tokens,Content_Tokens
0,147,2014.08.13,관심병사 베테랑 되기,자고 일어나기 무섭게 관심병사들이 스스로 목숨을 끊는 사건이 연이어 터지고 있다 지난 12일 육군 28사단 소속 관심병사 2명이 휴가를 나와 동반자살했고 제3군 사령부 직할부대 관심병사도 자신의 총으로 스스로 목숨을 끊어 주위를 안타깝게 하고 있다 저승에서나마 이승에서 누리지 못한 자유를 누릴 수 있기를 바란다증권시장에서 증권주는 관심병사와 같은 존재였다 박스권이 갖힌 증시에서 먼저 노란색 견장을 단 조선 해운 건설주들 만큼이나 앞이 보이지 않기는 마찬가지였다 연이은 적자 행진에 대규모 구조조정 소식으로 신문지상을 도배할 뿐 별다른 기쁜 소식을 찾기 어려운 증권업종을 보는 시선은 안타까움이었다 증권사들도 관심병사들 만큼이나 자유가 없었던 탓일 것이다 정부가 증시를 억누르던 규제를 풀어 자유를 주자 전일12일 증권업종 주가는 날개를 달았다정부는 증시 가격제한폭을 단계적으로 현재의 두배인 상하 30까지 확대하겠다는 내용을 투자활성화대책에 포함시켰다 더불어 최경환 경제팀이 내수경기 부양을 천명하고 배당확대를 유도하는 대책들을 내놓으면서 증권주는 노란색 견장을 떼고 분대를 이끄는 베테랑 병사에게 주어지는 녹색 견장으로 바꿔달 기세다 증시를 이끄는 원톱이 될수 있겠다는 기대감까지 감돌고 있다증시 전문가들은 상하한가 가격제한폭을 확대하는 정책이 실제 시행되면 박스권에 갇힌 증시에도 큰 도움이 되리란 의견을 내놓는다 이런 기대감 덕분인지 전일 코스피도 우크라이나와 이라크 등 지정학적 우려 속에서도 이틀 연속 상승세를 탔다 금리를 내릴 것으로 예상하는 이달 14일 금융통화위원회의 결과가 오늘 증시에도 단비가 될지 관심이 쏠린다증권주 주가가 오른 것은 더 폭넓어진 선택의 자유와 정부의 관심 덕분이다 주식 시장 관계자들에게 자신이 매매하고 싶은 가격을 스스로 선택할 자유 기업이 쌓아놓은 유보금을 투자자나 근로자들에게 나눠주도록 유도하는 정부의 관심 이 두 가지 덕분에 증권업종 주가에도 모처럼 훈풍이 부는 것이다 여전히 현대증권의 희망퇴직 소식으로 우울한 나날은 계속되고 있지만 업종을 보는 시장의 시선이 달라지고 있는 것은 분명해 보인다관심병사가 배테랑 병사가 되는 것도 마찬가지일 것이다 다양한 임무 중에서 굳이 적성에 맞지 않는 임무를 관심병사에게 줄 필요가 없다 그들에게 선택할 자유를 주고 관심병사라는 호칭에 걸맞게 선임병사와 장교들이 세심한 관심을 기울인다면 증권주 주가처럼 그들도 자신감을 찾을 수 있을 것이다,https://www.edaily.co.kr/News/Read?newsId=01272646606186992&mediaCodeNo=257,"[관심, 병사, 베테랑, 되, 기]","[자, 고, 일어나, 기, 무섭, 게, 관심, 병사, 들, 이, 스스로, 목숨, 을, 끊, 는, 사건, 이, 연이, 어, 터지, 고, 있, 다, 지난, 12, 일, 육군, 28, 사단, 소속, 관심, 병사, 2, 명, 이, 휴가, 를, 나와, 동반, 자살, 했, 고제, 3, 군, 사령부, 직, 할, 부대, 관심, 병사, 도, 자신, 의, 총, 으로, 스스로, 목숨, 을, 끊, 어, 주위, 를, 안타깝, 게, 하, 고, 있, 다, 저승, 에서, 나마, 이승, 에서, 누리, 지, 못한, 자유, 를, 누릴, 수, 있, 기, 를, 바란다, 증권시장, 에서, 증권주, 는, 관심, 병사, 와, 같, 은, 존재, 였, 다, 박스권, 이, 갖힌, 증시, ...]"
1,148,2014.08.13,하나외환은행 PB프라이빗 뱅커통합브랜드 연내 출범하나,하나은행과 외환은행의 조기통합이 그룹 내 화두로 떠오른 가운데 가장 먼저 통합이 이뤄질 부문은 PB프라이빗 뱅커부문이 될 것으로 보인다 PB는 전문가 집단이라는 인식이 강해 다른 사업부문보다 통합에 대한 저항감이 덜한데다 두 은행의 시너지가 빨리 가시화될 수 있는 부문이다12일 하나금융지주에 따르면 하나금융은 올해 초 외환은행 인수 당시 하나은행 PB 사업본부와 외환은행 PB 통합을 목로 통합 브랜드 골드 클럽을 올해 출범시킬 예정이었다 다만 지난 6월 김정태 회장의 조기통합론 카드를 꺼내 들면서 애초 PB부문을 먼저 하나로 묶는다는 계획은 다소 차질을 빚고 있지만 그룹 내에선 PB부문 통합이 가장 단시간 내에 가장 효과를 낼 것이란 데는 공감대가 이뤄진 상황이다 올 상반기 현재 하나은행과 외환은행의 PB 관리자산은 각각 약 23조원 10조원 등이며 PB 인력은 각각 230명 103명 정도다 따라서 두 은행 PB부문이 통합하면 자산 33조원에 330명 이상의 대형 PB조직이 탄생하게 된다 지난 7월 하나은행과 외환은행의 조기통합 추진을 위한 결의문에서도 두 은행의 PB업무 외국환 경쟁력 등 강점 공유로 인한 시너지를 225억원으로 분석하기도 했다특히 하나은행 PB센터는 국내 최초로 지난 1995년 현대적 개념의 PB 비즈니스를 도입하는 등 업계에서 강자로 통한다 1997년 세무지원서비스 2005년 첫 해외PB파견 2006년 토탈 라이프케어 2011년 상속증여센터 등도 국내 최초로 도입했다 또 예치자산 1억원 이상의 VIP클럽 5억원 이상의 골드클럽 10억원 이상의 WM그룹으로 세분화 돼 있는데다 골드클럽과 WM그룹은 각각 센터수 15개 2개 PB인원도 51명 10명으로 은행권에서 강점을 지난 부문으로 평가 받고 있다 외환은행 PB 역시 외국매매와 해외 채널 등에서 강점이 있어 두 은행은 상호보완성 효과를 톡톡히 누릴 수 있다는 설명이다 외환은행 PB 부문은 5억원 이상의 WMC와 1억원 이상의 영업점 PB센터를 각각 10개 85개 운영 중인데 PB의 70 이상이 외국환관련 자격증을 가지고 있는데다 해외동포 등 비거주고객 조달 비중이 높고 해외교포 VIP 연계 가능성이 큰 장점이 있다금융권 관계자는 하나은행의 경우 국민은행 신한은행과 달리 10억원 이상의 고액자산가 비중이 높아 PB 1인당 생산성이 굉장히 높다며 외환은행의 기업 PB 고객에 대한 하나은행의 PB 서비스가 접목되면 다른 은행에 비해 확연히 높은 수익이 기대된다고 말했다 실제 두 은행의 PB부문이 통합이 조금씩 가시화되면서 은행권은 긴장하고 있다 저금리 기조가 장기화 되면서 은행들이 PB부문을 강화를 미래먹거리 중 하나로 강조하는 분위기 속에서 리테일과 외국환에 강점을 지닌데다 규모의 경제까지 갖춘 하나외환은행 통합 PB부문은 위협이 될 수 있기 때문이다한 시중은행 PB담당 임원은 현재 5억원 이상 자산가들을 영입하느라 각 은행이 WM센터를 특화하고 PB전용상품 개발에 매진하는 등 고액자산가 고객 확보전이 뜨겁다며 특히 하나은행이 10억원 이상 고객이 많아 외환은행 PB부문과 통합까지 이뤄지면 하나은행PB의 인기는 더욱 치솟을 것으로 보여 업계로선 긴장하지 않을 수 없다고 말했다,https://www.edaily.co.kr/News/Read?newsId=01141446606186992&mediaCodeNo=257,"[하나, 외환은행, PB, 프라이빗뱅커, 통합, 브랜드, 연내, 출범, 하, 나]","[하나은행, 과, 외환은행, 의, 조기, 통합, 이, 그룹, 내, 화두, 로, 떠오른, 가운데, 가장, 먼저, 통합, 이, 이뤄질, 부문, 은, PB, 프라이빗뱅커, 부문, 이, 될, 것, 으로, 보인다, PB, 는, 전문가, 집단, 이, 라는, 인식, 이, 강해, 다른, 사업부문, 보다, 통합, 에, 대한, 저항감, 이, 덜한, 데, 다, 두, 은행, 의, 시너지, 가, 빨리, 가시, 화, 될, 수, 있, 는, 부문, 이, 다, 12, 일, 하나금융지주, 에, 따르, 면, 하나, 금융, 은, 올해, 초, 외환은행, 인수, 당시, 하나은행, PB, 사업, 본부, 와, 외환은행, PB, 통합, 을, 목로, 통합, 브랜드, 골드, 클럽, 을, 올해, 출범, 시킬, 예정, 이, 었, 다, 다만, ...]"
2,149,2014.08.13,돈 되는 앱테크,천연화장품 애호가인 주부 김씨는 아이소이 앱을 통해 40 할인 쿠폰을 받았다 평소 아이소이 제품을 즐겨 사용했던 그는 앱 다운만으로 쿠폰을 받아서 뿌듯하다며 큰 할인

In [6]:
df.to_csv('News_Tokenized.csv', encoding='utf-8-sig')

## Part of Speech Tagging

In [7]:
def pos_tagging_text(tokens):
    return tokenizer.pos(" ".join(tokens))

df['Title_POS'] = df['Title_Tokens'].apply(pos_tagging_text)
df['Content_POS'] = df['Content_Tokens'].apply(pos_tagging_text)

df

In [ ]:
df.to_csv('News_POS_tagged.csv', encoding='utf-8-sig')

## Normalization